In [4]:
import numpy as np 
import pandas as pd 

C:\Users\singhy\AppData\Local\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (
C:\Users\singhy\AppData\Local\Temp\ipykernel_19080\200606040.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [5]:
data_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/raw_data"
output_dir = "C:/Users/singhy/Desktop/Chicago/cps_data/inflation/output"

In [6]:
# Data Sets 
lab_prod = pd.read_csv(f"{data_dir}/labor_data/labor_productivity_nonfarm.csv")
df = pd.read_csv(f"{data_dir}/labor_data/stocks.csv")
vacancy = pd.read_csv(f"{data_dir}/labor_data/vacancy.csv")
jolts_level = pd.read_csv(f"{data_dir}/labor_data/jolts_level.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:/Users/singhy/Desktop/Chicago/cps_data/inflation/raw_data/labor_data/labor_productivity_nonfarm.csv'

In [ ]:
d

In [3]:
vacancy

NameError: name 'vacancy' is not defined

In [20]:
jolts_level.columns = ['date', 'tot_quits', 'vacancy_stock', 'tot_hires', 'tot_layoffs']
jolts_level['date'] = pd.to_datetime(jolts_level['date'])

In [21]:
vacancy.columns = ['date', 'V', 'V_rate']  
vacancy = vacancy.iloc[8:].reset_index(drop=True)
vacancy = vacancy.dropna(subset=['date', 'V'])


In [7]:
vacancy.head(13)

NameError: name 'vacancy' is not defined

In [138]:
import pandas as pd
import numpy as np

# Create a precise mapping for decimal values to months with tolerance for minor differences
def map_to_month(decimal_date):
    fraction = decimal_date - int(decimal_date)
    
    # Define the exact mappings for decimal values to months with a small tolerance
    if np.isclose(fraction, 0.00, atol=0.01):
        month = 1  # January
    elif np.isclose(fraction, 0.08, atol=0.01):
        month = 2  # February
    elif np.isclose(fraction, 0.17, atol=0.01):
        month = 3  # March
    elif np.isclose(fraction, 0.25, atol=0.01):
        month = 4  # April
    elif np.isclose(fraction, 0.33, atol=0.01):
        month = 5  # May
    elif np.isclose(fraction, 0.42, atol=0.01):
        month = 6  # June
    elif np.isclose(fraction, 0.50, atol=0.01):
        month = 7  # July
    elif np.isclose(fraction, 0.58, atol=0.01):
        month = 8  # August
    elif np.isclose(fraction, 0.67, atol=0.01):
        month = 9  # September
    elif np.isclose(fraction, 0.75, atol=0.01):
        month = 10  # October
    elif np.isclose(fraction, 0.83, atol=0.01):
        month = 11  # November
    elif np.isclose(fraction, 0.92, atol=0.01):
        month = 12  # December
    else:
        raise ValueError(f"Fraction {fraction} does not match any month")
    
    return month

# Convert the decimal date to datetime
def convert_to_datetime(decimal_date):
    # Ensure the value is a float
    decimal_date = float(decimal_date)
    year = int(decimal_date)
    month = map_to_month(decimal_date)
    return pd.Timestamp(year=year, month=month, day=1)

# Apply the conversion function to your dataset
vacancy['date'] = vacancy['date'].apply(convert_to_datetime)

# Check the output
print(vacancy)


          date            V V_rate
0   1951-02-01  2295.689006   3.72
1   1951-03-01  2554.677439   4.09
2   1951-04-01  2502.487739   4.05
3   1951-05-01  2534.983959   4.09
4   1951-06-01  2392.152707   3.88
..         ...          ...    ...
842 2021-04-01         9193   5.71
843 2021-05-01         9483   5.89
844 2021-06-01        10185   6.32
845 2021-07-01        11098   6.88
846 2021-08-01        10439   6.46

[847 rows x 3 columns]


In [139]:
df['DATE'] = pd.to_datetime(df['DATE'])

# Convert the 'DATE' column to datetime format
lab_prod['DATE'] = pd.to_datetime(lab_prod['DATE'])

# Set the day to the 1st of each month
lab_prod['DATE'] = lab_prod['DATE'].dt.to_period('M').dt.to_timestamp()

# Set 'DATE' as the index
lab_prod.set_index('DATE', inplace=True)

# Resample to monthly frequency and forward fill missing values
lab_prod_monthly = lab_prod.resample('ME').ffill()

# Reset the index
lab_prod_monthly.reset_index(inplace=True)
lab_prod_monthly['DATE'] = lab_prod_monthly['DATE'].apply(lambda x: x.replace(day=1))


In [140]:
lab_prod

,OPHNFB
DATE,
1947-01-01,22.265
1947-04-01,22.771
1947-07-01,22.074
1947-10-01,23.002
1948-01-01,23.106
...,...
2023-04-01,109.631
2023-07-01,110.863
2023-10-01,111.827


In [141]:
data = df.merge(lab_prod_monthly, on='DATE', how='outer')  
data.rename(columns={
    'DATE': 'date',
    'PAYEMS': 'E',
    'UNEMPLOY': 'U',
    'CPIAUCSL': 'P', 
    'OPHNFB': 'Z' 
}, inplace=True)


data.rename(columns={
    'DATE': 'date',
    'PAYEMS': 'E',
    'UNEMPLOY': 'U',
    'CPIAUCSL': 'P',
    'UNRATE': 'u_rate',
    'LNS17100000': 'ue_flow',
    'CIVPART': 'lab_force_rate',
    'OPHNFB': 'lab_prod' 
}, inplace=True)

In [142]:
data['P'] = pd.to_numeric(data['P'], errors='coerce')

In [143]:
data.loc[:, 'P_12m_change'] = data['P'].pct_change(periods=12, fill_method = None) * 100
data.loc[:, 'P_1m_change_annualize'] =  ((1+ data['P'].pct_change(periods=1, fill_method = None)) ** 12 - 1 ) * 100
data.loc[:, 'P_3m_change_annualize'] =  ((1+ data['P'].pct_change(periods=3, fill_method = None)) ** 3 - 1 ) * 100

data = data.dropna(subset=['U'])

In [144]:
vacancy['V'] = vacancy['V'].astype('float')

In [145]:
vacancy

,date,V,V_rate
0,1951-02-01,2295.689006,3.72
1,1951-03-01,2554.677439,4.09
2,1951-04-01,2502.487739,4.05
3,1951-05-01,2534.983959,4.09
4,1951-06-01,2392.152707,3.88
...,...,...,...
842,2021-04-01,9193.000000,5.71
843,2021-05-01,9483.000000,5.89
844,2021-06-01,10185.000000,6.32
845,2021-07-01,11098.000000,6.88


In [146]:
temp = data.merge(vacancy, on='date', how = 'outer')

In [147]:
jolts_level

,date,tot_quits,vacancy_stock,tot_hires,tot_layoffs
0,2000-12-01,2882,5088,5426,2018
1,2001-01-01,3245,5234,5722,2220
2,2001-02-01,3053,5097,5303,1855
3,2001-03-01,3054,4762,5528,2133
4,2001-04-01,3163,4615,5204,1883
...,...,...,...,...,...
279,2024-03-01,3409,8355,5617,1601
280,2024-04-01,3452,7919,5615,1542
281,2024-05-01,3403,8230,5655,1678
282,2024-06-01,3214,7910,5248,1560


In [148]:
final = temp.merge(jolts_level, on = ['date'], how= "outer")

In [149]:
# Jolts data 
threshold_date = pd.to_datetime('2000-12-01')

final.loc[final['date'] >= threshold_date, 'V'] = final['vacancy_stock']

In [150]:
# Convert columns to numeric, forcing errors to NaN
final['V'] = pd.to_numeric(final['V'], errors='coerce')
final['E'] = pd.to_numeric(final['E'], errors='coerce')
final['U'] = pd.to_numeric(final['U'], errors='coerce')

In [151]:
final['V_rate'] = (final['V'] / (final['E'] + final['U']))*100
final['U_rate'] = (final['U'] / (final['E'] + final['U']))*100

In [152]:
final['tightness'] = final["V"] / final["U"]

In [153]:
final

,date,E,U,P,Z,P_12m_change,P_1m_change_annualize,P_3m_change_annualize,V,V_rate,tot_quits,vacancy_stock,tot_hires,tot_layoffs,U_rate,tightness
0,1939-01-01,29923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1939-02-01,30100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1939-03-01,30280,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1939-04-01,30094,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1939-05-01,30299,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1023,2024-04-01,158214,6492.0,313.207,112.639,3.357731,3.820226,3.450804,7919.0,4.807961,3452.0,7919.0,5615.0,1542.0,3.941569,1.219809
1024,2024-05-01,158430,6649.0,313.225,NaN,3.250210,0.068986,2.108497,8230.0,4.985492,3403.0,8230.0,5655.0,1678.0,4.027769,1.237780
1025,2024-06-01,158548,6811.0,313.049,NaN,2.975629,-0.672196,0.788986,7910.0,4.783532,3214.0,7910.0,5248.0,1560.0,4.118917,1.161357
1026,2024-07-01,158637,7163.0,313.534,NaN,2.923566,1.875058,0.313539,7673.0,4.627865,3277.0,7673.0,5521.0,1762.0,4.320265,1.071199


In [154]:
final.to_csv(f"{output_dir}/clean_stocks.csv", index=False)